In [1]:
%%capture
!pip install imgaug
!pip install ipywidgets
!jupyter nbextension enable --py widgetsnbextension
!jupyter nbextension install --py --user witwidget
!jupyter nbextension enable witwidget --user --py
!jupyter labextension install @jupyter-widgets/jupyterlab-manager 
!jupyter labextension list

In [2]:
#### Import Packges 
%matplotlib widget
import matplotlib.pyplot as plt 
from pathlib import Path
import matplotlib.patches as patch
from tqdm.notebook import tqdm 
import numpy as np 
import cv2
import pydicom as dc 
import pandas as pd 
import os
import imgaug
import random
from imgaug import augmenters as iaa
from imgaug.augmentables.bbs import BoundingBox
from torchvision.utils import draw_bounding_boxes


import torch 
from torch.utils.data import Dataset , DataLoader
import torchvision
import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_lightning.plugins import DDPPlugin

#from pytorch_lightning.plugins.training_type.ddp import DDPPlugin
import warnings 
def fxn():
    warnings.warn("deprecated", DeprecationWarning)

with warnings.catch_warnings():
    warnings.simplefilter("ignore")

## **Per-Processing :** <a class="anchor" id="Per-Processing"></a>
  - **Per-Processing Tasks** :
      * resize the images to reduce the compute weights 
      * convert the input data into .npy format for easy loading the DataLoader 
      * exploer some few images virtualization 
      * Store the Label ( X , Y , WIDTH , HEIGHT ) to predict Bounding Box 

In [3]:
### set the path 
Path_label = "../input/rsna-pneumonia-detection-challenge/stage_2_train_labels.csv"
Path_img =  "../input/rsna-pneumonia-detection-challenge/stage_2_train_images"

In [4]:
#### Read data CSV file labels 
label = pd.read_csv(Path_label).fillna(0) ### here to replace (Nan) with zeros
x_min = label["x"].iloc[0].item()                
print(x_min)
label.head(5)

0.0


,patientId,x,y,width,height,Target
0,0004cfab-14fd-4e49-80ba-63a80b6bddd6,0.0,0.0,0.0,0.0,0
1,00313ee0-9eaa-42f4-b0ab-c148ed3241cd,0.0,0.0,0.0,0.0,0
2,00322d4d-1c29-4943-afc9-b6754be640eb,0.0,0.0,0.0,0.0,0
3,003d8fa0-6bf1-40ed-b54c-ac657f8495c5,0.0,0.0,0.0,0.0,0
4,00436515-870c-4b36-a041-de91049b9ab4,264.0,152.0,213.0,379.0,1


In [5]:
### Virtualize few samples from the data
fig , axis = plt.subplots(2,2,figsize=(6,6))
counter_img = 4 ### index to image has (x ,y) > 0 
for row in range(2):
    for cols in range(2):
        ## get image path 
        Image_Id = label.patientId.iloc[counter_img]
        Path_full_img = os.path.join(Path_img,Image_Id) + ".dcm" ## file extenstion 
        read_img = dc.read_file(Path_full_img).pixel_array
        ### get the corrdinates to drwan retengcel 
        x = label.x.iloc[counter_img]
        y = label.y.iloc[counter_img]
        Width = label.width.iloc[counter_img]
        Height= label.height.iloc[counter_img]
        axis[row][cols].imshow(read_img,cmap="gray")
        rectengel = patch.Rectangle((x,y),Width,Height,linewidth=1,edgecolor="r",facecolor='none')
        axis[row][cols].add_patch(rectengel)
        counter_img +=1

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [6]:
#### explore few samples from the data 
"""
- we will read all the files dicom and store them in npy foramt for easy loading the in DataFolder Class
- resize the images into 128x128 to handke by the RAM 
- split the data in to train and val 
- compute mean and STD 
"""
sums , sums_square = 0,0 
normalizer = 128*128 
index_img = 0
train_id =[] ###check the numbr samples in train and val 
val_id = []
Save_path = "./Processed_img"
for index_img in tqdm(range(len(label.patientId))):
    ### get the path image 
    patientId = label.patientId.iloc[index_img]
    full_path_img = os.path.join(Path_img,patientId) + ".dcm"
    ### read the image Pixel_array and resize 
    Image = dc.read_file(full_path_img).pixel_array
    resize_img = (cv2.resize(Image,(128,128)) / 255).astype(np.float32)
    ### Splite the data train 24000 and val 6300
    train_or_val = "train" if index_img < 24000 else "val"
    ### create the folder Val and train 
    if train_or_val =="train":
        train_id.append(patientId)
    else:
        val_id.append(patientId)
   ### save the array images into the folder 
    Save_path_img = Path( Save_path + "/" + train_or_val)
    Save_path_img.mkdir(parents=True,exist_ok=True)
    np.save(os.path.join(Save_path_img,patientId),resize_img)
    ### compute mean and std after 
    if train_or_val == "train":
        sums += np.sum(resize_img) / normalizer
        sums_square += (resize_img **2).sum() / normalizer

  0%|          | 0/30227 [00:00<?, ?it/s]

In [7]:
#### save the trainng and validation samples Path 
np.save("Processed-Heart-detetection-train",train_id)
np.save("Processed-Heart-detetection-val",val_id)

In [8]:
load_train = np.load("./Processed-Heart-detetection-train.npy")
load_train[:2]

array(['0004cfab-14fd-4e49-80ba-63a80b6bddd6',
       '00313ee0-9eaa-42f4-b0ab-c148ed3241cd'], dtype='<U36')

In [9]:
## compute mean nd STD to use them in augmentation setp
mean = sums / len(train_id)
std = np.sqrt((sums_square / len(train_id)) - mean**2)

In [10]:
mean , std

(0.4894162220594784, 0.24611802771185273)

## **Loading data and augmenetation :** <a class="anchor" id="Loading data and augmenetation"></a>
  - **Loading data and augmenetation Tasks** :
      * creat class Cardiac from torch Datasets functional helpers
      * augemente Bounding Box and Images
      


In [11]:
### creat Class Dataset 
class Cradiac_(Dataset):
    def __init__(self, path_label_csv , path_patientId , Root_path_Image , transform=None):
        self.path_label = pd.read_csv(path_label_csv).fillna(0)
        self.patientId = np.load(path_patientId)
        self.root_path = Root_path_Image
        self.transform = transform
        
    ### here we will need to return lenght of images 
    def __len__(self):
        return len(self.patientId)
    ### getItem by index to label image 
    def __getitem__(self,idx):
        ###check the image ID match with with label 
        patient_Id = self.patientId[idx]
        label_ = self.path_label[self.path_label["patientId"] == patient_Id]
        ### extract the corrdinates label and getItem value isung Item()
        x_min = self.path_label["x"].iloc[idx].item()
        y_min = self.path_label["y"].iloc[idx].item()
        x_max = x_min + self.path_label["width"].iloc[idx].item()
        y_max = y_min + self.path_label["height"].iloc[idx].item()
        Boxes = [x_min,y_min,x_max,y_max]
        ### get the image path to apply augemnetation step 
        image_path = os.path.join(self.root_path,patient_Id) + '.npy'
        load_img = np.load(image_path).astype(np.float32) 
        ### augment Bounding Box 
        if self.transform:
            Bounding_label = BoundingBox(x1=Boxes[0] , y1=Boxes[1], x2=Boxes[2] ,y2=Boxes[3] , label=None)
            img , BoundingBoxs = self.transform(load_img,Bounding_label)
            #### BoundingBoxes is tuple (x,y,x1,y1) of tow demission we need to index 
            corrdinates_boxes = BoundingBoxs[0][0], BoundingBoxs[0][1], BoundingBoxs[1][0], BoundingBoxs[1][1]
        
            #### load the image_ and BoundingBoxes to tensos 
            image_out= (img - 0.489) / 0.246
            image_out_ = torch.tensor(image_out).unsqueeze(0)
            box = torch.tensor(corrdinates_boxes)
            
            return image_out_ , box 

In [12]:
###  make class Augementation with list probs 
class Sequence_Aug(object):
    def __init__(self,Augmentation_commands, Proba=0.5):
        self.augmentation = Augmentation_commands #### this is list of augmentes instruction 
        self.probs = Proba ### this Var is for storing Probabilty values 
    def __call__(self, load_img , Bounding_label ):
        for i , tranform_aug in enumerate(self.augmentation):
            if type(self.probs) == list:
                prob_ = self.probs[i]
            else:
                prob_ = self.probs
            if  prob_ < 1 : # here chekc if the random proba avereg between [1-0]
                ## set the random Seed 
                image_ , boxes_ = tranform_aug(images=load_img ,bounding_boxes = Bounding_label )
        return  image_ , boxes_ 

In [13]:
augmentation_list = [iaa.GammaContrast(),
                     iaa.Affine(scale=(0.8,1.2),
                                rotate=(-10.10),
                                translate_px=(-10,10))]
transforms = Sequence_Aug(augmentation_list,0.5)


In [14]:
patient_id = './Processed-Heart-detetection-train.npy'
path_img_proccsed = "./Processed_img/train"
dataset = Cradiac_(Path_label,patient_id,path_img_proccsed,transform=transforms)

In [15]:
img_pro, box = dataset[0]

### plot img 
fig , axis = plt.subplots(1,1,figsize=(5,5))
axis.imshow(img_pro[0],cmap="gray")
rectengel = patch.Rectangle((box[0],box[1]),box[2],box[3],linewidth=2,edgecolor="r",facecolor='none')
axis.add_patch(rectengel)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## **Traning and building the model** <a class="anchor" id="Traning and building"></a>
   * **Step work pipline Pytorch lighiting** 
        - Loading the data from Cracdiac_ calss costume data Obejct
        - set the Hyper-parameters of DataLoader      
        - **Pipline model** Building model <a class="anchor" id="Section3"></a>
            - by calling this API torchvision 
              * torchvision.models.resnet152()
                * change the number of channel Conv1 from 3 to 1
                   * **(conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)**
                * change the output of last layer fully connected layer to 4 ==>  [x_min,y_min,x_max,y_max]
                   * **(fc): Linear(in_features=2048, out_features=1000, bias=True)**
                - **Notation** :  <div class="alert alert-info">since the competition is ended 3 years ago here i just tried give a briefy implementation of<strong> Pytorch **Lighiting**</strong></div> <br>
                - **that's why we will just run the Model on 10 Epochs and you can raise it to wished value if you would like**



In [16]:
### the set necesseray Path 
Path_training = "./Processed_img/train"
Path_validation = './Processed_img/val'
Path_label_csv_file = "../input/rsna-pneumonia-detection-challenge/stage_2_train_labels.csv"
Patient_Id_train = "./Processed-Heart-detetection-train.npy"
Patient_Id_val = "./Processed-Heart-detetection-val.npy"

In [17]:
### set the parameters Augmenration in list  and DataLoader parameters 
augmentation_ = [iaa.GammaContrast(),
                     iaa.Affine(scale=(0.8,1.2),
                                rotate=(-5.5),
                                translate_px=(-5,5))]
Transforms = Sequence_Aug(augmentation_,0.6)
batch_size = 64
num_workers = 2 
### loading thr dataset from Costume Class 
Data_traning = Cradiac_(Path_label_csv_file,Patient_Id_train,Path_training,transform=Transforms)
traning_set=DataLoader(Data_traning,batch_size = batch_size ,num_workers=num_workers,shuffle=True)
Data_validation  = Cradiac_(Path_label_csv_file,Patient_Id_val,Path_validation,transform=None)
validation_set = DataLoader(Data_traning,batch_size = batch_size ,num_workers=num_workers,shuffle=False)

In [18]:
import tensorflow as tf

In [19]:
### Building Models 
###### Creating the model 
class Detector_Cradiac(pl.LightningModule):
    def __init__(self):
        super().__init__()
        ### the PL of model 
        self.model = torchvision.models.resnet18()
        self.model.conv1 = torch.nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
        self.model.fc = torch.nn.Linear(in_features=512, out_features=4)
        ### setup the Optimizer and loss functions 
        self.Optimizer = torch.optim.Adam(self.model.parameters(),lr = 1e-4)
        self.Loss_F = torch.nn.MSELoss()
         # simple accuracy computation
#         self.train_acc = torchmetrics.Accuracy()
#         self.val_acc = torchmetrics.Accuracy()
        
    def forward(self, input_):
        Predicted_label = self.model(input_)
        return Predicted_label
        
    def training_step(self,batch , batch_idx):
        Image , label = batch 
        label = label.float()
        Predicted_label = self(Image)
        loss = self.Loss_F(Predicted_label,label)
        
        self.log("Train loss",loss)
            
        return loss
    
   
    #######
    ### here we did the same as Traiing PL we changed only the input_data disttro
    #######
    def validation_step(self,batch , batch_idx):
        Image , label = batch 
        label = label.float()
        Predicted_label = self(Image)
        loss = self.Loss_F(Predicted_label,label)
        
        self.log("Val loss",loss)
            
        return loss,
    
    def configure_optimizers(self):
        #Caution! You always need to return a list here (just pack your optimizer into one :))
        return [self.Optimizer]     

In [20]:
Model_ = Detector_Cradiac() ### instanace the model from the class 
Check_Point_Callbacks = ModelCheckpoint(
    monitor="Val loss", 
    save_top_k=12,
    mode="min")


In [21]:
# Create the trainer
# Change the gpus parameter to the number of available gpus on your system. Use 0 for CPU training

gpus = 2 #TODO
Trainer = pl.Trainer(accelerator='gpu', devices=2, logger=TensorBoardLogger(save_dir= "./processed/logs_weights_ex3"), log_every_n_steps=1,
                     callbacks=Check_Point_Callbacks,                    
                     max_epochs=10)

In [22]:
Trainer.fit(Model_,traning_set,validation_set)

/opt/conda/lib/python3.7/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:221: UserWarning: num_workers>0, persistent_workers=False, and strategy=ddp_spawn may result in data loading bottlenecks. Consider setting persistent_workers=True (this is a limitation of Python .spawn() and PyTorch)
  "num_workers>0, persistent_workers=False, and strategy=ddp_spawn"
/opt/conda/lib/python3.7/site-packages/pytorch_lightning/trainer/connectors/logger_connector/result.py:539: PossibleUserWarning: It is recommended to use `self.log('Val loss', ..., sync_dist=True)` when logging on epoch level in distributed setting to accumulate the metric across devices.
  category=PossibleUserWarning,


Training: 0it [00:00, ?it/s]

[W reducer.cpp:1282] Warning: find_unused_parameters=True was specified in DDP constructor, but did not find any unused parameters in the forward pass. This flag results in an extra traversal of the autograd graph every iteration,  which can adversely affect performance. If your model indeed never has any unused parameters in the forward pass, consider turning this flag off. Note that this warning may be a false positive if your model has flow control causing later iterations to have unused parameters. (function operator())
[W reducer.cpp:1282] Warning: find_unused_parameters=True was specified in DDP constructor, but did not find any unused parameters in the forward pass. This flag results in an extra traversal of the autograd graph every iteration,  which can adversely affect performance. If your model indeed never has any unused parameters in the forward pass, consider turning this flag off. Note that this warning may be a false positive if your model has flow control causing later 

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]